# DATASET EXTRACTION

## Import datasets and github
* Clone chestxray dataset from the github link https://github.com/ieee8023/covid-chestxray-dataset.git

* RSNA Pneumonia Detection Challenge dataset
https://www.kaggle.com/c/rsna-pneumonia-detection-challenge and unzip rsna_dataset here

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
! unzip -q '/content/drive/MyDrive/Medical_data_lias/data/data.zip' -d '/content/data/'

In [ ]:
! pip install pydicom einops
import numpy as np
import torch
import pandas as pd
import os
import random 
from shutil import copyfile
import pydicom as dicom
import cv2

     |████████████████████████████████| 1.9MB 12.8MB/s 


# Requirements

In [ ]:
!git clone https://github.com/iliasprc/COVIDNet.git
! pip install -r ./COVIDNet/requirements.txt

Cloning into 'COVIDNet'...
remote: Enumerating objects: 321, done.
remote: Counting objects: 100% (126/126), done.
remote: Compressing objects: 100% (92/92), done.
remote: Total 321 (delta 52), reused 88 (delta 31), pack-reused 195
Receiving objects: 100% (321/321), 48.43 MiB | 7.75 MiB/s, done.
Resolving deltas: 100% (141/141), done.
     |████████████████████████████████| 645kB 17.6MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from COVIDNet.model import *
model = CovidNet('large', n_classes=3).cuda()

from byol_pytorch import BYOL
from torchvision import models

# resnet = models.resnet50(pretrained=True).cuda()

# learner = BYOL(
#     resnet,
#     image_size = 224,
#     hidden_layer = 'avgpool'
# )

learner = BYOL(
    model,
    image_size = 224,
    hidden_layer = 'fc2'
)


opt = torch.optim.Adam(learner.parameters(), lr=1e-4)

def sample_unlabelled_images():
    return torch.randn(20, 3, 224, 224).cuda()

for _ in range(100):
    images = sample_unlabelled_images()
    loss = learner(images)
    opt.zero_grad()
    loss.backward()
    print(loss.item())
    opt.step()
    learner.update_moving_average() # update moving average of target encoder

# save your improved network
torch.save(model.state_dict(), '/content/drive/MyDrive/improved-covidnet_large.pt')

     |████████████████████████████████| 645kB 10.2MB/s 
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
4.030460834503174
3.4249939918518066
2.907473564147949
2.646979570388794
2.434429168701172
2.3089208602905273
2.2373123168945312
2.169525623321533
2.0579822063446045
2.0725438594818115
2.0720160007476807
2.00467848777771
2.032719850540161
1.9828532934188843
1.9661684036254883
1.9378807544708252
1.9296118021011353
1.9446099996566772
1.9372478723526
1.9428333044052124
1.9213488101959229
1.937448501586914
1.9325401782989502
1.9210811853408813
1.9230632781982422
1.860763430595398
1.9078744649887085
1.9210035800933838
1.8808990716934204
1.8772472143173218
1.8945121765136719
1.8930076360702515
1.9356788396835327
1.881738305091858
1.8791011571884155
1.8927844762802124
1.8716400861740112
1.9257099628448486
1.9031906127929688
1.9055709838867188
1.8867477178573608
1.8833383321762085
1.897756814956665
1.915903925895691
1.876418

# Training

In [ ]:
%cd COVIDNet
! python main.py --help
! python main.py #--dataset_name='COVIDx' --model='COVIDNet_small' --root_path='/content/data/data' --save='/content/drive/MyDrive/Medical_data_lias/saved_covid_cpkts'

[Errno 2] No such file or directory: 'COVIDNet'
/content/COVIDNet
usage: main.py [-h] [--batch_size BATCH_SIZE] [--log_interval LOG_INTERVAL]
               [--dataset_name DATASET_NAME] [--nEpochs NEPOCHS]
               [--device DEVICE] [--seed SEED] [--classes CLASSES] [--lr LR]
               [--weight_decay WEIGHT_DECAY] [--cuda] [--tensorboard]
               [--resume PATH]
               [--model {COVIDNet_small,resnet18,mobilenet_v2,densenet169,COVIDNet_large}]
               [--opt {sgd,adam,rmsprop}] [--root_path ROOT_PATH]
               [--save SAVE]

optional arguments:
  -h, --help            show this help message and exit
  --batch_size BATCH_SIZE
                        batch size for training
  --log_interval LOG_INTERVAL
                        steps to log.info metrics and loss
  --dataset_name DATASET_NAME
                        dataset name COVIDx or COVID_CT
  --nEpochs NEPOCHS     total number of epochs
  --device DEVICE       gpu device
  --seed SEED        